In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.pipeline import Pipeline

In [2]:
# Data import
rt = pd.read_csv('reviews_rt_all.csv', sep = '|')
imdb = pd.read_csv('imdb_small.csv', sep = '|')

In [ ]:
import os
import numpy
#BASE_DIR = '../'
TEXT_DATA_DIR = ''#BASE_DIR  + '../data/test'
TEXT_DATA_FILE_1 = "rt-polarity_neg.txt"
TEXT_DATA_FILE_2 = "rt-polarity_pos.txt"
HEADER = True

def load_data():
    x = []
    y = []
    for i in [TEXT_DATA_FILE_1, TEXT_DATA_FILE_2]:
        with open(os.path.join(TEXT_DATA_DIR, i), "r", encoding='utf-8', errors='ignore') as f:
            if HEADER:
                _ = next(f)
            if i[-7:-4] == "pos":
                temp_y = 1
            else: temp_y = 0
            for line in f:
                x.append(line.rstrip("\n"))
                y.append(temp_y)

    return x, y


data, labels = load_data()
data = pd.DataFrame({'label': labels, 'text': data})

In [32]:
df = pd.read_csv("amazon_all_train.csv", sep = '|')
df_bad = df[df.label == 0]
df_good = df[df.label == 1]
df_good_new = df_good.sample (n=185376, random_state=42)
result = pd.concat([df_good_new, df_bad])
result = result.sample(frac=1)

In [23]:
print (len(result.label))
print (len(result.text))

370752
370752


In [35]:
X_train_am, X_test_am, y_train_am, y_test_am = train_test_split(result.text, result.label, test_size=0.2, random_state=42, stratify=result.label)


In [25]:
# Split RT and IMDB datasets seapretely
X_train_rt, X_test_rt, y_train_rt, y_test_rt = train_test_split(rt.text, rt.label, test_size=0.2, random_state=42, stratify=rt.label)
X_train_imdb, X_test_imdb, y_train_imdb, y_test_imdb = train_test_split(imdb.text, imdb.label, test_size=0.2, random_state=42, stratify=imdb.label)

# Then concatenate
X_train = pd.concat([X_train_rt, X_train_imdb])
X_test = pd.concat([X_test_rt, X_test_imdb])
y_train = pd.concat([y_train_rt, y_train_imdb])
y_test = pd.concat([y_test_rt, y_test_imdb])

In [28]:
# Take the last 22 words from each review in the train set
#X_train = X_train.str.split().apply(lambda x:  ' '.join(x for x in x[-22:]))

In [14]:
# Stopwords
STOPWORDS = ['by','does', 'was', 'were', 'the', 'of', 'end', 'and', 'is']    

In [34]:
cvect = CountVectorizer()
counts = cvect.fit_transform(X_train_am)

classifier = PassiveAggressiveClassifier(C=0.001, fit_intercept = False, shuffle = False, n_iter = 91, n_jobs = -1)
pipeline = Pipeline([('vectorizer', CountVectorizer(binary=True,ngram_range=(1,4),stop_words=STOPWORDS)), ('classifier', classifier)])
model = pipeline.fit(X=X_train_am, y=y_train_am)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [43]:
# Compare Validation Accuracy on RT, IMDB and mixed test sets
y_pred_rt = model.predict(X_test_rt)
y_pred_imdb = model.predict(X_test_imdb)
y_pred_am = model.predict(X_test_am)
y_pred = model.predict(X_test)

print ("Accuracy RT :", metrics.accuracy_score(y_test_rt, y_pred_rt))
print ("Accuracy IMDB :", metrics.accuracy_score(y_test_imdb, y_pred_imdb))
print ("Accuracy RT+IMDB :", metrics.accuracy_score(y_test, y_pred))
print ("Accuracy Am :", metrics.accuracy_score(y_test_am, y_pred_am))

Accuracy RT : 0.812396452587
Accuracy IMDB : 0.9034
Accuracy RT+IMDB : 0.842212174825


In [44]:
joblib.dump(pipeline, 'output.pkl')

['output.pkl']